In [1]:
# =============================================================================
# # 1. SETUP: IMPORTING LIBRARIES
# =============================================================================
import torch
import cv2 # OpenCV for image handling
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# =============================================================================
# # 2. LOAD THE PRE-TRAINED YOLOv5 MODEL
# =============================================================================
# Load the YOLOv5 model from PyTorch Hub. 'yolov5s' is the small, fast version.
# 'pretrained=True' downloads the weights trained on the COCO dataset.
try:
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have an active internet connection to download the model.")
    exit()

print("--- YOLOv5 Model Loaded Successfully ---")
# Set confidence threshold and IOU threshold for filtering detections
model.conf = 0.4  # Confidence threshold (0-1)
model.iou = 0.45  # Intersection over Union threshold (0-1)
print(f"Confidence threshold set to {model.conf}")
print("\n" + "="*50 + "\n")

Using cache found in C:\Users\RIYA/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-9-6 Python-3.12.2 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


--- YOLOv5 Model Loaded Successfully ---
Confidence threshold set to 0.4




In [3]:
# =============================================================================
# # 3. LOAD IMAGE AND RUN DETECTION
# =============================================================================
# Define the path to your image
IMAGE_PATH = 'img2.jpeg' # IMPORTANT: Change this to your image's filename

# Load the image using OpenCV
try:
    img = cv2.imread(IMAGE_PATH)
    # YOLO expects images in RGB format, but OpenCV loads them in BGR. We need to convert.
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
except Exception as e:
    print(f"Error loading image: {e}")
    print(f"Please make sure '{IMAGE_PATH}' is in the correct directory.")
    exit()

print(f"--- Running detection on {IMAGE_PATH} ---")
# Pass the image to the model to get detections
results = model(img_rgb)

# The results contain information about bounding boxes, confidence scores, and class labels.
results.print()

C:\Users\RIYA/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


--- Running detection on img2.jpeg ---


image 1/1: 183x275 7 persons
Speed: 11.4ms pre-process, 217.0ms inference, 18.2ms NMS per image at shape (1, 3, 448, 640)


In [4]:
# =============================================================================
# # 4. PROCESS AND VISUALIZE THE RESULTS
# =============================================================================
# The esults can be converted to a pandas DataFrame for easier access
results_df = results.pandas().xyxy[0]

print("--- Detected Objects ---")
print(results_df)

# Loop through the detected objects and draw boxes on the original image
for index, row in results_df.iterrows():
    # Get coordinates, class name, and confidence
    x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
    confidence = row['confidence']
    class_name = row['name']
    
    # Draw the bounding box
    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2) # Blue box with thickness 2
    
    # Create the label text
    label = f'{class_name}: {confidence:.2f}'
    
    # Put the label text above the box
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

print("\n--- Bounding boxes and labels drawn on the image ---")

# Display the final image using Matplotlib
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('YOLOv5 Object Detection Result', fontsize=16)
plt.axis('off')
plt.savefig('object_detection_result.png') # Save the final image for LinkedIn
plt.show()

--- Detected Objects ---
         xmin        ymin        xmax        ymax  confidence  class    name
0   50.343658   99.007294   60.595615  131.628403    0.773227      0  person
1  114.002296  105.395607  126.196594  140.542633    0.654427      0  person
2  224.003464   93.580391  233.144501  121.054848    0.621082      0  person
3  234.701279   97.121605  242.098648  120.077690    0.617597      0  person
4   43.219467   99.361710   49.797241  116.175270    0.574704      0  person
5  247.790512   99.314079  258.393768  128.263565    0.483406      0  person
6   65.034935  100.868523   70.630127  114.840164    0.410876      0  person

--- Bounding boxes and labels drawn on the image ---
